In [18]:
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth

In [19]:
#Hugging Face Login:
#from huggingface_hub import login
#from google.colab import userdata
#hf_token=userdata.get('hugging-key')
#login(hf_token)
#print(hf_token)

In [20]:
!pip install hf
!hf download unsloth/DeepSeek-R1-Distill-Llama-8B --local-dir ./model

Fetching 15 files: 100% 15/15 [00:00<00:00, 2098.76it/s]
Download complete: : 0.00B [00:00, ?B/s]              /content/model
Download complete: : 0.00B [00:00, ?B/s]


In [21]:
from unsloth import FastLanguageModel
from google.colab import userdata
import torch
# 资源条件>=24GPU 可选择14/32B
max_seq_length = 2048
dtype = None
load_in_4bit = True
hf_token=userdata.get('hugging-key')
model, tokenizer = FastLanguageModel.from_pretrained(
   #   model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B",
   model_name = "./model",
      max_seq_length = max_seq_length,
      dtype = dtype,
    #  load_in_4bit = load_in_4bit,
    #  token = hf_token
)

==((====))==  Unsloth 2026.2.1: Fast Llama patching. Transformers: 5.0.0.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.494 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

Unsloth: Will load ./model as a legacy tokenizer.


In [22]:
#Prompt Template (Inference):
prompt_style = """以下是描述任务的说明，并附有提供进一步背景的输入。
  请写一个恰当的回应，完成该任务。
  在回答之前，请仔细思考问题，并创建一个逐步的思维链，以确保逻辑清晰且准确。

  ### 任务描述:
  您是一位在临床推理、诊断和治疗计划方面具有高级知识的医学专家。
  请回答以下医学问题。

  ### 问题:
  {}

  ### 回答:
  <think>{}"""

In [23]:
 #Question & Inference:
question = "一位61岁的女性，有长期的咳嗽或打喷嚏等活动中不自主漏尿的历史，但夜间没有漏尿。她接受了妇科检查和Q-tip试验。根据这些检查结果，膀胱容漏尿。她接受了妇科检查和Q-tip试验。根据这些检查结果，膀胱容量测定（膀胱压力-容积曲线）最有可能显示她的残余尿量和膀胱逼尿肌的收缩情况是什么？"
#question = "I feel painful in my knee"

FastLanguageModel.for_inference(model)
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 2048)

<｜begin▁of▁sentence｜>以下是描述任务的说明，并附有提供进一步背景的输入。
  请写一个恰当的回应，完成该任务。
  在回答之前，请仔细思考问题，并创建一个逐步的思维链，以确保逻辑清晰且准确。

  ### 任务描述:
  您是一位在临床推理、诊断和治疗计划方面具有高级知识的医学专家。
  请回答以下医学问题。

  ### 问题:
  一位61岁的女性，有长期的咳嗽或打喷嚏等活动中不自主漏尿的历史，但夜间没有漏尿。她接受了妇科检查和Q-tip试验。根据这些检查结果，膀胱容漏尿。她接受了妇科检查和Q-tip试验。根据这些检查结果，膀胱容量测定（膀胱压力-容积曲线）最有可能显示她的残余尿量和膀胱逼尿肌的收缩情况是什么？

  ### 回答:
  <think>
嗯，这个问题看起来有点复杂，不过我还是试着一步步理清楚。首先，患者是一个61岁的女性，有长期的咳嗽或打喷嚏导致不自主的漏尿历史，但夜间没有漏尿。所以，这可能意味着她的膀胱容量在白天有所下降，或者夜间的膀胱压力较低，不容易溢出。

接下来，她接受了妇科检查和Q-tip试验。Q-tip试验通常用于诊断尿道功能障碍，特别是前列腺问题，但也可以用于膀胱功能评估。通过Q-tip试验，医生可以测量膀胱的压力-容积曲线，这能反映膀胱的排尿能力和容量。

膀胱容量测定通常包括膀胱压力-容积曲线的绘制。这个曲线可以显示当膀胱被充盈时的压力，以及随着膀胱容积的增加，压力的变化。通常，膀胱的容积在充盈时会增加，而压力则会随着容积的增加而升高，形成一个曲线。

在这种情况下，患者的残余尿量和膀胱逼尿肌的收缩情况可以通过膀胱压力-容积曲线来观察。膀胱容量测定通常会测量膀胱的最大容积和压力。残余尿量指的是膀胱在排尿前仍然保留的尿液量，而膀胱逼尿肌的收缩情况则与膀胱的排尿压力有关。

考虑到患者夜间没有漏尿，可能她的膀胱在夜间的容积或压力较低，或者夜间的排尿压力不足以促使膀胱排出尿液。因此，膀胱压力-容积曲线可能显示出在夜间膀胱的容积较低，或者在膀胱充盈时的压力较低，这可能反映了膀胱的功能障碍或受损。

总的来说，膀胱容量测定将帮助医生了解患者的膀胱功能，包括残余尿量和膀胱的排尿肌收缩情况，从而制定相应的治疗计划。可能的治疗措施包括膀胱健忘法、药物治疗或手术干预，以改善膀胱功能，减少不自主的漏尿。
</think>

根据

In [24]:
#Training Prompt Template:
train_prompt_style = """以下是描述任务的说明，并附有提供进一步背景的输入。
  请写一个恰当的回应，完成该任务。
  在回答之前，请仔细思考问题，并创建一个逐步的思维链，以确保逻辑清晰且准确。

  ### 任务描述:
  您是一位在临床推理、诊断和治疗计划方面具有高级知识的医学专家。
  请回答以下医学问题。

  ### 问题:
  {}

  ### 回答:
  <think>
  {}
  </think>
  {}"""

In [25]:
#Data Formatting Function:
EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN
# 数据集格式化函数
def formatting_prompts_func(examples):
      inputs = examples["Question"]
      cots = examples["Complex_CoT"]
      outputs = examples["Response"]
      texts = []
      for input, cot, output in zip(inputs, cots, outputs):
          text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
          texts.append(text)
      return{
          "text": texts,
      }


In [26]:
#Load Dataset (加载数据集):
# 加载数据集
from datasets import load_dataset
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT","zh", split = "train[0:20000]")
#dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "zh")
#print(f"Dataset size: {len(dataset['train'])}")
print(dataset)

dataset = dataset.map(formatting_prompts_func, batched = True,)
print(dataset)
dataset["text"][0]

Dataset({
    features: ['Question', 'Complex_CoT', 'Response'],
    num_rows: 20000
})
Dataset({
    features: ['Question', 'Complex_CoT', 'Response', 'text'],
    num_rows: 20000
})


'以下是描述任务的说明，并附有提供进一步背景的输入。\n  请写一个恰当的回应，完成该任务。\n  在回答之前，请仔细思考问题，并创建一个逐步的思维链，以确保逻辑清晰且准确。\n\n  ### 任务描述:\n  您是一位在临床推理、诊断和治疗计划方面具有高级知识的医学专家。\n  请回答以下医学问题。\n\n  ### 问题:\n  根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？\n\n  ### 回答:\n  <think>\n  这个小孩子在夏天头皮上长了些小结节，一直都没好，后来变成了脓包，流了好多脓。想想夏天那么热，可能和湿热有关。才一岁的小孩，免疫力本来就不强，夏天的湿热没准就侵袭了身体。\n\n用中医的角度来看，出现小结节、再加上长期不愈合，这些症状让我想到了头疮。小孩子最容易得这些皮肤病，主要因为湿热在体表郁结。\n\n但再看看，头皮下还有空洞，这可能不止是简单的头疮。看起来病情挺严重的，也许是脓肿没治好。这样的情况中医中有时候叫做禿疮或者湿疮，也可能是另一种情况。\n\n等一下，头皮上的空洞和皮肤增厚更像是疾病已经深入到头皮下，这是不是说明有可能是流注或瘰疬？这些名字常描述头部或颈部的严重感染，特别是有化脓不愈合，又形成通道或空洞的情况。\n\n仔细想想，我怎么感觉这些症状更贴近瘰疬的表现？尤其考虑到孩子的年纪和夏天发生的季节性因素，湿热可能是主因，但可能也有火毒或者痰湿造成的滞留。\n\n回到基本的症状描述上看，这种长期不愈合又复杂的状况，如果结合中医更偏重的病名，是不是有可能是涉及更深层次的感染？\n\n再考虑一下，这应该不是单纯的瘰疬，得仔细分析头皮增厚并出现空洞这样的严重症状。中医里头，这样的表现可能更符合‘蚀疮’或‘头疽’。这些病名通常描述头部严重感染后的溃烂和组织坏死。\n\n看看季节和孩子的体质，夏天又湿又热，外邪很容易侵入头部，对孩子这么弱的免疫系统简直就是挑战。头疽这个病名听起来真是切合，因为它描述的感染严重，溃烂到出现空洞。\n\n不过，仔细琢磨后发现，还有个病名似乎更为合适，叫做‘蝼蛄疖’，这病在中医里专指像这种严重感染并伴有深部空洞的情况。它也涵盖了化脓和皮肤增厚这些症状。\n\n哦，该不会是夏季湿热，导致湿毒入侵，孩

In [27]:
#设置模型参数
model = FastLanguageModel.get_peft_model(
      model,
      r=16,
      target_modules=[
          "q_proj",
          "k_proj",
          "v_proj",
          "o_proj",
          "gate_proj",
          "up_proj",
          "down_proj",
      ],
      lora_alpha=16,
      lora_dropout=0.1,
      bias="none",
      use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
      random_state=3407,
      use_rslora=False,
      loftq_config=None,
  )

In [28]:
#设置训练（微调）参数
from trl import SFTTrainer, SFTConfig
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
      model=model,
      tokenizer=tokenizer,
      train_dataset=dataset,
      args=SFTConfig(
          dataset_text_field="text",
          max_seq_length=max_seq_length,
          dataset_num_proc=2,
          per_device_train_batch_size=2,
          gradient_accumulation_steps=4,
          warmup_steps=5,
         # max_steps=60,
           max_steps=1000,
          learning_rate=2e-4,
          fp16=not is_bfloat16_supported(),
          bf16=is_bfloat16_supported(),
          logging_steps=10,
          optim="adamw_8bit",
          weight_decay=0.01,
          lr_scheduler_type="linear",
          seed=3407,
          output_dir="outputs",
      ),
  )

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20,000 | Num Epochs = 1 | Total steps = 1,000
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Step,Training Loss
10,2.125832
20,1.623949
30,1.551010
40,1.534442
50,1.500091
60,1.463599
70,1.464120
80,1.467525
90,1.432036
100,1.476674


In [ ]:
#微调后验证
question = "一位61岁的女性，有长期的咳嗽或打喷嚏等活动中不自主漏尿的历史，但夜间没有漏尿。她接受了妇科检查和Q-tip试验。根据这些检查结果，膀胱容漏尿。她接受了妇科检查和Q-tip试验。根据这些检查结果，膀胱容量测定（膀胱压力-容积曲线）最有可能显示她的残余尿量和膀胱逼尿肌的收缩情况是什么？"
#question = "I feel painful in my knee. reply in chinese"

FastLanguageModel.for_inference(model)  # Unsloth has 2x faster inference!
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 2048)

In [ ]:
#Save locally (保存到本地):
# 保存到本地
new_model_local = "DeepSeek-R1-Medical-Doctor"
model.save_pretrained(new_model_local)
tokenizer.save_pretrained(new_model_local)

model.save_pretrained_merged(new_model_local, tokenizer, save_method = "merged_16bit",)

In [ ]:
#Push to HuggingFace (推送到huggingface仓库):
# 推送到huggingface仓库
#new_model_online = "mustangai/DeepSeek-R1-Medical-Doctor"
#model.push_to_hub(new_model_online)
#tokenizer.push_to_hub(new_model_online)

#model.push_to_hub_merged(new_model_online, tokenizer, save_method = "merged_16bit")